In [ ]:
#one day i may move the functionality to start and monitor servers into this document, 
# but for now, start up everything with helao_interface.py before using this code
import requests
import json
from matplotlib import pyplot
url = "http://192.168.31.114:13380"
#url = "http://127.0.0.1:13380"
import time
import ipywidgets
import numpy
import pickle
import asyncio
import websockets
sub_data = [] #holds data sent to the notebook by the orchestrator

In [ ]:
#live data routines
#loop is websocket connection to receive data from the orchestrator
#update plot live plots this data as it comes in
#init_history_plot and load_figure are subroutines of update_plot
#much of this is adapted from https://stackoverflow.com/questions/68918341/live-update-several-plots-in-jupyter-notebook


uri = "ws://127.0.0.1:13374/ws"

async def loop(var_x,var_y): # non-blocking coroutine, updates data source
    async with websockets.connect(uri) as ws:
        while True:
            new_data = await ws.recv()
            new_data = json.loads(new_data)
            if not isinstance(new_data,float):
                sub_data.append(new_data)
                update_plot(var_x,var_y)

def update_plot(var_x,var_y):
    with plot_wrapper:
        fig, axs = load_figure(template_fig)
        plot_wrapper.clear_output(wait=True)
        for i in range(len(axs)):
            axs[i].scatter([data[var_x[i]][0] for data in sub_data],[data[var_y[i]][0] for data in sub_data],s=1)
        pyplot.show()

### i need to start plotting of the data from index x
### 3 plots instaed of 1
        
def init_history_plot(var_x,var_y):
    """
    Create plot template (dump)
    
    Returns: pickled str
    """
    fig, axs = pyplot.subplots(nrows=1, ncols=len(var_x), figsize=(14,4))
    dat = pickle.dumps(fig)
    pyplot.close()
    return dat


def load_figure(dump):
    """
    Load Figure from dump
    
    Returns: (Figure, Axes)
    """
    # https://github.com/ipython/ipykernel/issues/231
    import ipykernel.pylab.backend_inline as back_inline
    import matplotlib.backends.backend_agg as back_agg
    back_inline.new_figure_manager_given_figure = back_agg.new_figure_manager_given_figure
    figure = pickle.loads(dump)
    # https://github.com/matplotlib/matplotlib/issues/17627/
    figure._cachedRenderer = None
    return figure, figure.axes

In [ ]:
#start
#send this command to initialize the orchestrator for normal single experiments
collectionkey = 'default'
experiment = {'soe':['orchestrator/start'],'params':{'start':{'collectionkey':collectionkey}},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})

In [ ]:
#finish
#send this command to shut down the orchestrator for those
experiment = {'soe':['orchestrator/finish'],'params':{'finish':None},'meta':{}}
requests.post(url+'/orchestrator/addExperiment',params = {'experiment':json.dumps(experiment)})

In [ ]:
#emergency functions
thread = 0
requests.post(url+'/orchestrator/clear',params = {'thread':thread})
#requests.post(url+'/orchestrator/kill',params = {'thread':thread})
#requests.post(url+'/orchestrator/pause',params = {'thread':thread})
#requests.post(url+'/orchestrator/resume',params = {'thread':thread})

In [ ]:
#get data and start a live plotting instance
#this sets up a websocket connection with the orchestrator and begins live plotting of received spectra
%matplotlib inline
asyncio.create_task(loop(["t_s","Ewe_V","Z_real"],["Ewe_V","I_A","Z_imag"]))
plot_wrapper = ipywidgets.Output()
template_fig = init_history_plot(["t_s","Ewe_V","Z_real"],["Ewe_V","I_A","Z_imag"])
plot_interface = ipywidgets.Box(children=[plot_wrapper])
plot_interface

In [ ]:
#default plotting method for single spectra
def plot(data,var_x,var_y):
    pyplot.plot([data[var_x][0] for data in sub_data],[data[var_y][0] for data in sub_data])
    pyplot.show()

In [ ]:
sub_data